# Project Machine Learning Pipeline - good Emotion Prediction

In [1]:
#import library
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pandas as pd

## Set variabel

In [2]:
PIPELINE_NAME = "emotion-pipeline"
SCHEMA_PIPELINE_NAME = "emotion-tfdv-schema"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [3]:
emotion = pd.read_csv("data/emotion.csv")
emotion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254539 entries, 0 to 254538
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   text     254539 non-null  object
 1   emotion  254539 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.9+ MB


In [4]:
DATA_ROOT = 'data'

In [5]:
interactive_context = InteractiveContext(pipeline_root = PIPELINE_ROOT)

## Data Ingestion

### Examplegen

- Dirancang untuk membagi dataset menjadi dua kelompok utama, yaitu data pelatihan (train) dan data evaluasi (eval), menggunakan komponen CsvExampleGen di TensorFlow Extended (TFX). Dalam konfigurasi output, proses ini memanfaatkan objek example_gen_pb2.Output dengan penyesuaian hash buckets untuk masing-masing split, yaitu "train" dengan 8 hash buckets dan "eval" dengan 2 hash buckets. Implementasi ini memastikan pembagian dataset yang seimbang dan acak, mempersiapkan data yang akan digunakan pada tahap-tahap selanjutnya dalam pengembangan pipeline machine learning dengan TFX.

In [8]:
## membagi data train dan eval
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits = [
        example_gen_pb2.SplitConfig.Split(name = "train", hash_buckets = 8),
        example_gen_pb2.SplitConfig.Split(name = "eval", hash_buckets = 2)
    ])
)

example_gen = CsvExampleGen(input_base = DATA_ROOT, output_config = output)

In [9]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

### StatisticGen

- Komponen ini, yang diwakili oleh variabel statistic_gen, bertanggung jawab untuk menghasilkan statistik deskriptif dari dataset yang sudah dibagi sebelumnya. Informasi ini mencakup ringkasan statistik untuk setiap fitur dalam dataset, seperti mean, std dev, dan nilai-nilai kuantitatif lainnya. Hasil dari komponen ini menjadi input bagi langkah-langkah berikutnya dalam proses Data Validation.

In [10]:
statistic_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)

interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [11]:
interactive_context.show(statistic_gen.outputs['statistics'])

### SchemaGen

- SchemaGen menggunakan output dari StatisticsGen untuk menghasilkan skema dataset. Skema ini mencakup definisi tentang tipe data, range nilai yang diharapkan, serta properti dan batasan lainnya untuk setiap fitur. Skema ini memberikan panduan tentang bagaimana setiap fitur seharusnya berperilaku dalam dataset. Hasil dari SchemaGen digunakan sebagai input untuk langkah selanjutnya.

In [12]:
schema_gen = SchemaGen(statistics = statistic_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [13]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'emotion',INT,required,,-
'text',BYTES,required,,-


### ExampleValidator

- Komponen ini, yang diwakili oleh variabel example_validator, menggunakan statistik dari StatisticsGen dan skema dari SchemaGen untuk mengevaluasi apakah dataset mematuhi skema yang telah ditentukan. Jika ada ketidaksesuaian antara data aktual dan skema yang diharapkan, ExampleValidator memberikan informasi yang berguna tentang jenis dan lokasi ketidaksesuaian tersebut. Langkah ini membantu memastikan kekonsistenan dan kualitas dataset yang digunakan dalam pengembangan model.

In [14]:
example_validator = ExampleValidator(
    statistics = statistic_gen.outputs["statistics"],
    schema = schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [15]:
interactive_context.show(example_validator.outputs["anomalies"])

## Data Preprocessing

### Transform

-  Terdapat proses data preprocessing yang dilakukan menggunakan komponen Transform dari TensorFlow Extended (TFX). Sebuah modul transformasi yang disebut "emotion_transform.py" dibuat untuk mengubah dataset sebelum digunakan dalam pelatihan model machine learning. Modul ini mengimplementasikan fungsi preprocessing_fn yang mengonversi label ("emotion") menjadi tipe data int64 dan mengubah teks ("text") menjadi huruf kecil. Selanjutnya, komponen Transform menjalankan proses transformasi ini dengan menggunakan input dari output examples dari CsvExampleGen dan schema dari SchemaGen. Hasil transformasi, yang diberi nama khusus, kemudian disimpan dan dapat digunakan sebagai input untuk tahap-tahap berikutnya dalam pengembangan pipeline machine learning. Eksekusi proses transformasi ini dilakukan menggunakan InteractiveContext, memastikan bahwa dataset telah diolah dan disesuaikan sebelum digunakan dalam pelatihan model selanjutnya.

In [17]:
TRANSFORM_MODULE_FILE = "emotion_transform.py"

In [20]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf

LABEL_KEY = "emotion"
FEATURE_KEY = "text"

def transformed_name(key) :
    return key + "_xf"

def preprocessing_fn(inputs) :
    outputs = {}
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    return outputs

Overwriting emotion_transform.py


In [21]:
transform = Transform(
    examples = example_gen.outputs["examples"],
    schema = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Transform\transform_graph\6\.temp_path\tftransform_tmp\25420e2ef7364227b45acbcc4010bd6f\assets


INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Transform\transform_graph\6\.temp_path\tftransform_tmp\25420e2ef7364227b45acbcc4010bd6f\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Model Development

-  fokus pada pengembangan model menggunakan TensorFlow Extended (TFX). Pertama, modul emotion_trainer.py ditulis untuk mendefinisikan fungsi-fungsi yang dibutuhkan dalam proses pelatihan dan evaluasi model. Dalam proses ini, dilakukan pengolahan teks, pembuatan model machine learning dengan lapisan vektorisasi teks, dan penggunaan TensorFlow Transform untuk memproses fitur-fitur dataset. Selanjutnya, komponen Trainer digunakan untuk melatih model dengan mengambil output dari komponen Transform. Resolusi model dilakukan menggunakan Resolver dengan strategi "LatestBlessedModelStrategy", memastikan penggunaan model terbaru yang telah divalidasi. Proses evaluasi model menggunakan Evaluator dengan konfigurasi metrik tertentu, seperti AUC, False Positives, True Positives, dan Binary Accuracy. Hasil evaluasi ditampilkan melalui TensorFlow Model Analysis (TFMA) untuk memberikan wawasan terperinci tentang performa model. Langkah-langkah ini bersama-sama membentuk tahap pengembangan model yang komprehensif dan terintegrasi dalam pipeline machine learning dengan TFX.

In [35]:
TRAINER_MODULE_FILE = "emotion_trainer.py"

In [36]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "emotion"
FEATURE_KEY = "text"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset
 
# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing emotion_trainer.py


### Trainer

In [37]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file = os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(splits = ['train']),
    eval_args = trainer_pb2.EvalArgs(splits = ['eval'])
)

interactive_context.run(trainer)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 64)                1088  

INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Trainer\model\7\Format-Serving\assets


1000/1000 [==============================] - 9s 8ms/step - loss: 0.2290 - binary_accuracy: 0.8891 - val_loss: 0.1429 - val_binary_accuracy: 0.9464
Epoch 2/10
 997/1000 [============================>.] - ETA: 0s - loss: 0.1332 - binary_accuracy: 0.9491
Epoch 2: val_binary_accuracy improved from 0.94639 to 0.95030, saving model to pipelines\emotion-pipeline\Trainer\model\7\Format-Serving
INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Trainer\model\7\Format-Serving\assets


1000/1000 [==============================] - 8s 8ms/step - loss: 0.1333 - binary_accuracy: 0.9491 - val_loss: 0.1271 - val_binary_accuracy: 0.9503
Epoch 3/10
 991/1000 [============================>.] - ETA: 0s - loss: 0.1232 - binary_accuracy: 0.9507
Epoch 3: val_binary_accuracy improved from 0.95030 to 0.95323, saving model to pipelines\emotion-pipeline\Trainer\model\7\Format-Serving
INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Trainer\model\7\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\emotion-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Resolver

In [38]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type = Model),
    model_blessing = Channel(type = ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Evaluator

In [42]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs = [tfma.ModelSpec(label_key = 'emotion')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics = [
                tfma.MetricConfig(class_name = 'ExampleCount'),
                tfma.MetricConfig(class_name = 'AUC'),
                tfma.MetricConfig(class_name = 'FalsePositives'),
                tfma.MetricConfig(class_name = 'TruePositives'),
                tfma.MetricConfig(class_name = 'FalseNegatives'),
                tfma.MetricConfig(class_name = 'TrueNegatives'),
                tfma.MetricConfig(class_name = 'BinaryAccuracy',
                    threshold = tfma.MetricThreshold(
                        value_threshold = tfma.GenericValueThreshold(
                            lower_bound = {'value' : 0.5}
                    ),
                    change_threshold = tfma.GenericChangeThreshold(
                        direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute = {'value' : 0.0001}
                    )
                )
            )
        ])
    ]
)

#### Make Matrix Evaluation

In [44]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config = eval_config
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [45]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Model Deployment

- Model Deployment menggunakan komponen Pusher dalam TensorFlow Extended (TFX). Dalam proses ini, Pusher melakukan pendorongan model yang telah dilatih dan divalidasi ke sistem deployment dengan konfigurasi yang ditentukan. Model hasil pelatihan dari komponen Trainer dan model blessing dari komponen Evaluator digunakan sebagai input, dan pendorongan dilakukan ke direktori tertentu, dalam hal ini "serving_model_dir/emotion-prediction-model". Proses pendorongan ini memastikan bahwa model yang akan dideploy telah melewati validasi dengan baik sebelum dirilis ke lingkungan produksi atau sistem yang dituju. Dengan menjalankan komponen Pusher, notebook.ipynb ini menyelesaikan siklus pengembangan dan deployment model secara terintegrasi dalam pipeline TFX.

### Pusher

In [53]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model = trainer.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = "serving_model_dir/emotion-prediction-model"
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 16
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))